# Introductory Bayesian Data Analysis 

![title](results\figures\presentation\1.png)

**Valerio Bonometti                                                                                    
PhD Candidate University of York**

## Agenda

- Introduction

- One
    * From counts to probability
    * Bayesian updating
    * Priors, Likelyhood and Posterior 
  
- Two
    * Parameters Estimation
    * Bivariate Linear Model

- Three
    * Generalized Linear Model
    * Hierarchical Models
    

In [30]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.stats import binom, beta

import seaborn as sns

sns.set(style='white', font_scale=1.5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from modules.visuals import visualize_sp_update